In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
# from catboost import CatBoostregressor

In [2]:
df = pd.read_csv('data/raw.csv')

In [3]:
X = df.drop(columns=['math_score'],axis=1)

In [4]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [5]:
y = df['math_score']

In [6]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,cat_features),
        ("StandardScaler",numeric_transformer,num_features)
    ]
)

In [7]:
X = preprocessor.fit_transform(X)

In [8]:
X.shape

(1000, 19)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(800, 19)
(200, 19)
(800,)
(200,)


In [10]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square =r2_score(true, predicted)
    return mae, rmse, r2_square

In [11]:
models= {
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train,y_train_pred)

    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test,y_test_pred) 

    print('***********************************************************')
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for Training set')
    print('- Mean Absolute Error: {:.4f}'.format(model_train_mae))
    print('- Root Mean Squared Error: {:.4f}'.format(model_train_rmse))
    print('- R2 score: {:.4f}'.format(model_train_r2))
    
    print('-----------------------------------------------------------')

    print('Model performance for Test set')
    print('- Mean Absolute Error: {:.4f}'.format(model_test_mae))
    print('- Root Mean Squared Error: {:.4f}'.format(model_test_rmse))
    print('- R2 score: {:.4f}'.format(model_test_r2))
    r2_list.append(model_test_r2)
    


***********************************************************
Linear Regression
Model performance for Training set
- Mean Absolute Error: 4.2715
- Root Mean Squared Error: 5.3293
- R2 score: 0.8740
-----------------------------------------------------------
Model performance for Test set
- Mean Absolute Error: 4.2222
- Root Mean Squared Error: 5.4252
- R2 score: 0.8790
***********************************************************
Lasso
Model performance for Training set
- Mean Absolute Error: 5.2063
- Root Mean Squared Error: 6.5938
- R2 score: 0.8071
-----------------------------------------------------------
Model performance for Test set
- Mean Absolute Error: 5.1579
- Root Mean Squared Error: 6.5197
- R2 score: 0.8253
***********************************************************
Ridge
Model performance for Training set
- Mean Absolute Error: 4.2650
- Root Mean Squared Error: 5.3233
- R2 score: 0.8743
-----------------------------------------------------------
Model performance for Test s

In [12]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"])

,Model Name,R2_Score
4,Decision Tree,0.726492
3,K Neighbors Regressor,0.783813
1,Lasso,0.825320
6,XGBRegressor,0.827797
7,AdaBoost Regressor,0.842790
5,Random Forest Regressor,0.851786
0,Linear Regression,0.879046
2,Ridge,0.880593
